In [10]:
import os

file_path = "/Users/macmontxinho/Desktop/Teams/Wyscout/2RFEFv1.csv"

# Tamaño en bytes
file_size_bytes = os.path.getsize(file_path)

# Tamaño legible en KB y MB
file_size_kb = file_size_bytes / 1024
file_size_mb = file_size_kb / 1024

print(f"Tamaño del archivo:")
print(f"- {file_size_bytes} bytes")
print(f"- {file_size_kb:.2f} KB")
print(f"- {file_size_mb:.2f} MB")

Tamaño del archivo:
- 2004536 bytes
- 1957.55 KB
- 1.91 MB


In [11]:
import pandas as pd

file_path = "/Users/macmontxinho/Desktop/Teams/Wyscout/2RFEFv1.csv"

df = pd.read_csv(file_path, sep=";")  # Asegúrate del separador
rows, cols = df.shape

print(f"📊 El archivo tiene {rows} filas y {cols} columnas.")

📊 El archivo tiene 2546 filas y 130 columnas.


In [12]:
import pandas as pd

df = pd.read_csv("/Users/macmontxinho/Desktop/Teams/Wyscout/2RFEFv1.csv", sep=";")
print(df.shape)

(2546, 130)


In [13]:
import ast

def is_list_str(x):
    try:
        return isinstance(ast.literal_eval(x), list)
    except:
        return False

cols_with_list_strings = [col for col in df.columns if df[col].apply(is_list_str).any()]
print("🔍 Columnas con listas codificadas como strings:", cols_with_list_strings)

🔍 Columnas con listas codificadas como strings: ['passport_country_codes', 'positions', 'passport_country_names']


### Modificamos passport_country_names

In [14]:
import ast

def limpiar_passports(x):
    try:
        lista = ast.literal_eval(x)
        if isinstance(lista, list):
            return "/".join(lista) if lista else "desconocido"
        else:
            return "desconocido"
    except Exception:
        return "desconocido"

df["passport_country_names"] = df["passport_country_names"].apply(limpiar_passports)

### Eliminamos 'passport_country_codes', 'positions'

In [15]:
# Eliminar columnas innecesarias
df.drop(columns=["passport_country_codes", "positions"], inplace=True, errors="ignore")

In [16]:
df.shape

(2546, 128)

### Identificamos duplicados

In [17]:
# Identificar duplicados completos (todas las columnas)
duplicados = df[df.duplicated(keep=False)]

# Exportar duplicados a Excel
ruta_salida = "/Users/macmontxinho/Desktop/Teams/Wyscout/Duplicados_2RFEF_Wyscout.xlsx"
duplicados.to_excel(ruta_salida, index=False)

print(f"🔁 Se han encontrado {duplicados.shape[0]} filas duplicadas y exportado a:\n{ruta_salida}")

🔁 Se han encontrado 684 filas duplicadas y exportado a:
/Users/macmontxinho/Desktop/Teams/Wyscout/Duplicados_2RFEF_Wyscout.xlsx


In [18]:
# Contar filas antes
original_count = len(df)

# Eliminar duplicados (exactamente iguales en todas las columnas)
df = df.drop_duplicates(keep="first")

# Contar filas después
final_count = len(df)
removed = original_count - final_count

# Mostrar resumen
print(f"📊 Filas originales: {original_count}")
print(f"✅ Filas tras eliminar duplicados: {final_count}")
print(f"🗑️ Filas eliminadas: {removed}")

📊 Filas originales: 2546
✅ Filas tras eliminar duplicados: 2204
🗑️ Filas eliminadas: 342


### Posiciones que tienen número

In [19]:
import re

def clean_position(value):
    if isinstance(value, str):
        only_letters = re.sub(r'[^A-Za-z]', '', value)
        return only_letters if only_letters else "desconocido"
    return "desconocido"

for col in ["primary_position", "secondary_position", "third_position"]:
    df[col] = df[col].apply(clean_position)

### Traducción de posiciones

In [20]:
# Aplicar traducciones de posiciones a DataFrame `df`

# Diccionarios de traducción
posiciones_castellano = {
    'AMF': 'Mediocampista Ofensivo', 'CB': 'Defensa Central', 'CF': 'Delantero Centro',
    'DMF': 'Mediocampista Defensivo', 'GK': 'Portero', 'LAMF': 'Mediocampista Ofensivo Izquierdo',
    'LB': 'Lateral Izquierdo', 'LCB': 'Defensa Central Izquierdo', 'LCMF': 'Mediocampista Central Izquierdo',
    'LDMF': 'Mediocampista Defensivo Izquierdo', 'LW': 'Extremo Izquierdo', 'LWB': 'Carrilero Izquierdo',
    'LWF': 'Delantero Extremo Izquierdo', 'RAMF': 'Mediocampista Ofensivo Derecho', 'RB': 'Lateral Derecho',
    'RCB': 'Defensa Central Derecho', 'RCMF': 'Mediocampista Central Derecho', 'RDMF': 'Mediocampista Defensivo Derecho',
    'RW': 'Extremo Derecho', 'RWB': 'Carrilero Derecho', 'RWF': 'Delantero Extremo Derecho'
}

posiciones_generales = {
    'GK': 'Portero', 'LB': 'Lateral', 'RB': 'Lateral', 'LWB': 'Lateral', 'RWB': 'Lateral',
    'CB': 'Central', 'LCB': 'Central', 'RCB': 'Central',
    'DMF': 'Pivote', 'LDMF': 'Pivote', 'RDMF': 'Pivote',
    'LCMF': 'Mediocampista', 'RCMF': 'Mediocampista',
    'AMF': 'Mediapunta', 'LAMF': 'Mediapunta', 'RAMF': 'Mediapunta',
    'LW': 'Extremo', 'RW': 'Extremo',
    'CF': 'Delantero', 'RWF': 'Delantero', 'LWF': 'Delantero'
}

# Limpiar número y normalizar texto en posiciones
import re

def limpiar_posicion(pos):
    if isinstance(pos, str):
        letras = re.sub(r'[^A-Za-z]', '', pos).upper()
        return letras if letras else "Desconocido"
    return "Desconocido"

# Aplicar limpieza
for col in ["primary_position", "secondary_position", "third_position"]:
    df[col] = df[col].apply(limpiar_posicion)

# Agregar traducciones
for i, col in enumerate(["primary_position", "secondary_position", "third_position"], start=1):
    df[f"{col}_ESP"] = df[col].map(posiciones_castellano).fillna("Desconocido")
    df[f"position_gen_ESP{i}"] = df[col].map(posiciones_generales).fillna("Desconocido")

In [ ]:
# 1. Traducir columna "foot"
traduccion_pie = {
    'right': 'Diestro',
    'left': 'Zurdo',
    'both': 'Ambidiestro'
}

# Limpiar y traducir columna 'foot'
df["foot"] = df["foot"].fillna("Desconocido").astype(str).str.strip().str.lower()
df["Pie"] = df["foot"].map(traduccion_pie).fillna("Desconocido")

In [ ]:
# 2. Normalizar columnas string clave
columnas_str = ["current_team_name", "last_club_name", "birth_country_name", "birth_country_code", "full_name", "name"]

for col in columnas_str:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace(["", "nan", "None", "0"], "Desconocido")

In [23]:
# 3. Validar fechas y convertir a datetime
for date_col in ["birth_date", "contract_expires"]:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# 3.1 (Opcional) Calcular edad exacta
if "birth_date" in df.columns:
    today = pd.Timestamp.today()
    df["calculated_age"] = df["birth_date"].apply(lambda d: today.year - d.year - ((today.month, today.day) < (d.month, d.day)) if pd.notnull(d) else None)


### Ordenamos la columna

In [24]:
# Definir orden base original
columns_order = [
    "id", "name", "full_name", "image", "birth_date", "birth_day", "birth_country_name", "birth_country_code",
    "passport_country_names", "age", "height", "weight", "foot", "positions",
    "primary_position", "primary_position_percent", "secondary_position", "secondary_position_percent", "third_position",
    "third_position_percent", "current_team_name", "current_team_logo", "current_team_color", "last_club_name", "on_loan",
    "market_value", "contract_expires", "domestic_competition_name", "total_matches", "minutes_on_field",
    "goals", "goals_avg", "non_penalty_goal", "non_penalty_goal_avg", "head_goals", "head_goals_avg",
    "shots", "shots_avg", "shots_on_target_percent", "goal_conversion_percent", "assists", "assists_avg", "xg_shot",
    "xg_shot_avg", "xg_assist", "xg_assist_avg", "shot_assists_avg", "pre_assist_avg", "pre_pre_assist_avg",
    "crosses_avg", "accurate_crosses_percent", "cross_from_left_avg", "successful_cross_from_left_percent",
    "cross_from_right_avg", "successful_cross_from_right_percent", "cross_to_goalie_box_avg", "dribbles_avg",
    "successful_dribbles_percent", "offensive_duels_avg", "offensive_duels_won", "duels_avg", "duels_won",
    "defensive_duels_avg", "defensive_duels_won", "aerial_duels_avg", "aerial_duels_won",
    "successful_attacking_actions_avg", "touch_in_box_avg", "progressive_run_avg", "accelerations_avg",
    "received_pass_avg", "received_long_pass_avg", "foul_suffered_avg", "fouls_avg", "yellow_cards",
    "yellow_cards_avg", "red_cards", "red_cards_avg", "passes_avg", "accurate_passes_percent",
    "forward_passes_avg", "successful_forward_passes_percent", "back_passes_avg", "successful_back_passes_percent",
    "vertical_passes_avg", "successful_vertical_passes_percent", "short_medium_pass_avg",
    "accurate_short_medium_pass_percent", "long_passes_avg", "successful_long_passes_percent", "average_pass_length",
    "average_long_pass_length", "smart_passes_avg", "accurate_smart_passes_percent", "key_passes_avg",
    "passes_to_final_third_avg", "accurate_passes_to_final_third_percent", "pass_to_penalty_area_avg",
    "accurate_pass_to_penalty_area_percent", "through_passes_avg", "successful_through_passes_percent",
    "deep_completed_pass_avg", "deep_completed_cross_avg", "progressive_pass_avg", "successful_progressive_pass_percent",
    "tackle_avg", "possession_adjusted_tackle", "shot_block_avg", "interceptions_avg",
    "possession_adjusted_interceptions", "conceded_goals", "conceded_goals_avg", "shots_against", "shots_against_avg",
    "clean_sheets", "save_percent", "xg_save", "xg_save_avg", "prevented_goals", "prevented_goals_avg",
    "back_pass_to_gk_avg", "goalkeeper_exits_avg", "gk_aerial_duels_avg", "free_kicks_taken_avg",
    "direct_free_kicks_taken_avg", "direct_free_kicks_on_target_percent", "corners_taken_avg", "penalties_taken",
    "penalties_conversion_percent"
]

# Insertar nuevas columnas justo después de las correspondientes
custom_additions = {
    "foot": ["Pie"],
    "primary_position": ["primary_position_ESP", "position_gen_ESP1"],
    "secondary_position": ["secondary_position_ESP", "position_gen_ESP2"],
    "third_position": ["third_position_ESP", "position_gen_ESP3"],
    "birth_date": ["calculated_age"]
}

# Construir nuevo orden incluyendo columnas extra al lado de las originales
final_order = []
for col in columns_order:
    final_order.append(col)
    if col in custom_additions:
        final_order.extend(custom_additions[col])

# Añadir al final cualquier columna restante no incluida
remaining_cols = [c for c in df.columns if c not in final_order]
final_order += remaining_cols

# Reordenar el DataFrame
df = df[[col for col in final_order if col in df.columns]]

In [25]:
df.to_excel("/Users/macmontxinho/Desktop/Teams/Wyscout/data/2RFEF_Limpio.xlsx", index=False)